In [171]:
from __future__ import print_function
import pandas as pd
import editdistance
from os import listdir
from os.path import isfile, join
import os
import numpy as np
import math
import yaml

In [172]:
with open("yaml/pages.yaml",'rU') as f:
    pages = yaml.load(f.read())

ScannerError: mapping values are not allowed here
  in "<string>", line 1623, column 51:
     ...  How birds go to roost and awake: Going to roost
                                         ^

In [ ]:
pages[243]

In [173]:
# note that the page values start at 1, so add a value to the start of these
# arrays
species_files = ['0']
species_names_headers = ['0']
web_chapters = ['0']
subchapters = ['0']
from_pages = ['0']
explores = ['0']


for i in range(1,len(pages)+1):
    d = pages[i]
    species_names_headers.append(d['title'])
    species_files.append(d['filename'])
    web_chapters.append(d['chapter_name'].replace(".",":"))
    subchapters.append(d['subchapter_name'].replace('"',"&quot;").replace('--','&mdash;'))
    from_pages.append(d['from_page'])
    explores.append(d['explores']) 
    

with open("lookup_php2.php", 'w') as f:
    f.write("<?php ")
    t = '$species_names_headers=["'+'","'.join(species_names_headers)+'"' + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))
    t = "$species_files=['"+"','".join(species_files)+"'" + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))    
    t = "$web_chapters=['"+"','".join(web_chapters)+"'" + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))   
    t = '$subchapters=["'+'","'.join(subchapters)+'"' + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))   
    t = "$from_pages=['"+"','".join(map(str,from_pages))+"'" + "];\n"
    f.write(t.encode("utf-8"))   
    t = "$explores=['"+"','".join(map(str,explores))+"'" + "];\n"
    f.write(t.encode("utf-8"))
    f.write(" ?>")

# Make Image Tables

In [174]:


image_template = '''			<div class="species-gallery">
				<div>
					<img src="img/noca2.jpg" alt="" />
					<p class="caption">Female; Photo by Marie Read</p>
				</div>
				<div>
					<img src="img/noca1.jpg" alt="" />
					<p class="caption">Male; Photo by Brian E. Small</p>
				</div>
				<div>
					<img src="img/noca2.jpg" alt="" />
					<p class="caption">Female; Photo by Marie Read</p>
				</div>
				
				
			</div>
'''

an_image = '''				<div>
					<img src="photos/{photo_file}.jpg" alt="" />
					<p class="caption">{caption}</p>
				</div>'''



for i in range(1,len(pages)+1):

    
    image_table_text = '<div class="species-gallery">'

    for photo in pages[i]['pictures']:
        photo_file = photo['photo_file']
        caption = photo['photo_caption']
        image_table_text += an_image.format(photo_file = photo_file ,
                                           caption = caption )

    image_table_text += "</div>"
        
#     if len(pages[i]['pictures'])>1:
#         print(pages[i]['pictures'], i)
    with open("Species_Image_HTML/"+pages[i]['filename']+"_images.html", 'w') as f:
        f.write(image_table_text)


In [175]:
for photo in pages[1]['pictures']:
    print(photo['photo_caption'], photo['photo_file'])

Photo by Brian E. Small BcCh-BS-1-Black-capped_Chickadee_B9H4324


In [176]:
# !mkdir Species_Image_HTML_test

# Search by Chapter Text

In [177]:


text = '''				<ul class="collapser">
					<li><a href="#"><i class="fas fa-plus-circle"></i> Chapter 2: Birds and Their Sounds</a>
						<ul class="show-hide collapser">
							<li><a href="#"><i class="fas fa-plus-circle"></i> Birds, sing and call</a>
								<ul class="show-hide">
									
									<li><a href="recording.php">Black-capped chickadee</a></li>
									<li><a href="recording.php">White-breasted nuthatch</a></li>
									<li><a href="recording.php">Veery</a></li>
									<li><a href="recording.php">American robin</a></li>
									<li><a href="recording.php">Gray catbird</a></li>
									<li><a href="recording.php">Common yellowthroat</a></li>
									
								</ul>
							</li>
							<li><a href="#"><i class="fas fa-plus-circle"></i> Birds without a song just call</a>
								<ul class="show-hide">
									
									<li><a href="recording.php">Canada goose</a></li>
									<li><a href="recording.php">Mallard</a></li>
									<li><a href="recording.php">Chimney swift</a></li>
									<li><a href="recording.php">Killdeer</a></li>
									<li><a href="recording.php">Double-crested cormorant</a></li>
									<li><a href="recording.php">Red-shouldered hawk</a></li>
									
								</ul>
							</li>
						</ul>
					</li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Chapter 3: Why and How Birds Sing</a>
						<ul class="show-hide collapser">
							<li><a href="#"><i class="fas fa-plus-circle"></i> Why sing?</a>
								<ul class="show-hide">
									
									<li><a href="recording.php">Eastern whip-poor-will, Mexican whip-poor-will</a></li>
									<li><a href="recording.php">Virginia rail, sora</a></li>
									<li><a href="recording.php">Brown thrasher</a></li>
									<li><a href="recording.php">Black-throated blue warbler</a></li>
									<li><a href="recording.php">Wild turkey</a></li>
									<li><a href="recording.php">Dickcissel</a></li>
									<li><a href="recording.php">Yellow-headed blackbird</a></li>
									
								</ul>
							</li>
						</ul>
					</li>
				</ul>
'''

chapter_list_text = u'<ul class="collapser">'

chapter_heading = u'''<li> <a href="#" id="{chapter_heading_forlink}"><i class="fas fa-plus-circle"></i> {chapter_heading}</a>
						<ul class="show-hide collapser">
                        '''

subchapter_heading = u'''<li> <a href="#" id="{subchapter_heading_forlink}"><i class="fas fa-plus-circle"></i> {subchapter_heading}</a>
								<ul class="show-hide">
                                '''

species_link = u'''<li><a href="recording.php?page={file_index}">{species_name}</a></li>'''

subchapter_footer = u'''								</ul>
							</li>
'''

chapter_footer = u'''						</ul>
					</li>'''


def webify_title(x):
    return x.replace(" ","_").replace(":","").replace(".","").replace(";","").replace(",","")

def add_chapter_heading(x):
    global chapter_list_text
    chapter_title = x['chapter_name']
    chapter_title_link = webify_title(chapter_title)
    chapter_list_text += chapter_heading.format(chapter_heading=chapter_title,
                                            chapter_heading_forlink = chapter_title_link)
    
def add_subchapter_heading(x):
    global chapter_list_text
    subchapter_title = x['subchapter_name']  
    subchapter_title_link = webify_title(subchapter_title)
    chapter_list_text += subchapter_heading.format(subchapter_heading = subchapter_title,
                                                   subchapter_heading_forlink=subchapter_title_link )

    
# initiate and declare global variables
last_chapter_title = pages[1]['chapter_name']
last_subchapter_title = pages[1]['subchapter_name']
chapter_title = last_chapter_title
subchapter_title = last_subchapter_title

add_chapter_heading(pages[1])
add_subchapter_heading(pages[1])


for i in range(1, len(pages)+1):
    chapter_title = pages[i]['chapter_name']
    subchapter_title = pages[i]['subchapter_name']
  
    if last_subchapter_title != subchapter_title:
        chapter_list_text += subchapter_footer
        
    if last_chapter_title != chapter_title:
        chapter_list_text += chapter_footer
        add_chapter_heading(pages[i])
        last_chapter_title = chapter_title
    
    if last_subchapter_title != subchapter_title:
        add_subchapter_heading(pages[i])
        last_subchapter_title = subchapter_title
    
    species_name = pages[i]['title']
    chapter_list_text += species_link.format(file_index = i, species_name = species_name)

        
    
        
chapter_list_text += subchapter_footer
chapter_list_text += chapter_footer


chapter_list_text += u"</ul>"

# print(chapter_list_text)

with open("chapter_lookup.php", 'w') as f:
    f.write(chapter_list_text.encode("utf-8"))

In [178]:
# pages[i]

In [179]:
!ls yaml

birds.yaml   explore.yaml pages.yaml


In [193]:
with open("yaml/birds.yaml",'rU') as f:
    birds = yaml.load(f.read())

In [194]:


# birds[0]

In [195]:
# pages

In [196]:
for p in pages:
    print(p)#,pages[p])
    break

1


In [197]:

species_example_text = '''				<ul class="collapser">
					<li><a href="#"><i class="fas fa-plus-circle"></i> Ducks, geese</a>
						<ul class="show-hide">
							<li><em>Anatidae</em>
								
								<ul>
									<li>Canada goose, <a href="recording.php">27-29</a></li>
									<li>American wigeon, <a href="recording.php">40</a></li>
									<li>Mallard, <a href="recording.php">30-31</a></li>
									<li>Ruddy duck, <a href="recording.php">64</a></li>
								</ul>
							</li>
						</ul>
						
					</li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Quail, chicken, grouse, turkey, chachalaca</a></li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Grebes, pigeons, doves</a></li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Nightjars</a></li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Thrushes</a></li>
					<li><a href="#"><i class="fas fa-plus-circle"></i> Mockingbirds, catbirds, thrashers, starlings</a>
						
						<ul class="show-hide">
							<li><em>Mimidae</em>
								
								<ul>
									<li>Northern mockingbird, <a href="recording.php">238</a>, <a href="recording.php">335-342</a>, <a href="recording.php">463</a>, <a href="recording.php">516-517</a></li>
								</ul>
							</li>
						</ul>
					</li>
				</ul>'''

species_list_text = u'<ul class="collapser">\n'

group_name_text = u'''					<li><a href="#" onclick='update_url("{update_url}#species-list")' id="{group_name_link}"><i class="fas fa-plus-circle"></i> {group_name}</a>
						<ul class="show-hide">
'''

family_name_text = u'''							<li><em> {family_name}</em>
								
								<ul>
'''

species_text_start = u'''									\
<li>{species_name}, '''

species_links = u'''<a href="recording.php?page={species_index}">{songs}</a>'''

species_text_end = u'''</li>
'''


family_name_footer = u'''								</ul>
							</li>'''

group_name_footer = u'''						</ul>
						
					</li>
'''



groups = []
for b in birds:
    if b['group'] not in groups:
        groups.append(b['group'])




# df_species = pd.read_excel('Excel Files with organizing data/Listen by species.xlsx')


def get_species_links(song, pages):
    song_numbers = str(song).split("-")
    p_min = int(song_numbers[0])
    for p in pages:
        numbers_file = pages[p]['songs'].split("-")
        numbers_file = np.array(map(int, numbers_file))
        if p_min >= numbers_file.min() and p_min <= numbers_file.max():
            species_index = p
    return species_index


for group_name in groups:

    
    species_list_text += group_name_text.format(group_name=group_name, 
                                                group_name_link=webify_title(group_name),
                                               update_url="?expand="+webify_title(group_name))
    
    families = []
    for b in birds:
        if b['group'] == group_name:
            if b['family'] not in families:
                families.append(b['family'])
    
    for family_name in families:        
        species_list_text += family_name_text.format(family_name = family_name )
        
        for b in birds:
            if b['family'] == family_name:
        
                species_name = b['bird_name']
                species_list_text += species_text_start.format(species_name=species_name)#

                # get the links
                songs = b['songs']
                links_to_add = []
                for song in songs:
                    species_index = get_species_links(song, pages)
                    links_to_add.append(species_links.format(species_index=species_index,songs=song))
                species_list_text += ",".join(links_to_add)         
                
        species_list_text += species_text_end
            
        species_list_text += family_name_footer
    
    species_list_text += group_name_footer


species_list_text += u"</ul>"

with open("species_lookup.php", 'w') as f:
    f.write(species_list_text.encode("utf-8"))

In [198]:
with open("yaml/explore.yaml",'rU') as f:
    explores = yaml.load(f.read())

In [199]:
explores[1]

{'a_title': '1.  Chick-a-dee calls of North American chickadees.',
 'chapter': '2. Birds and Their Sounds',
 'subchapter': 'Birds sing and call',
 'x_from_page': 8}

In [200]:
# note that the page values start at 1, so add a value to the start of these
# arrays
titles_e = ['0']
explore_files = ['0']
chapters_e = ['0']
subchapters_e = ['0']
from_pages_e = ['0']




for e in explores:
    index = e
    titles_e.append(explores[e]['a_title'])
    explore_files.append('{}.html'.format(index))
    chapters_e.append(explores[e]['chapter'])
    subchapters_e.append(explores[e]['subchapter'].replace('"',"&quot;").replace('--','&mdash;').replace("'","&apos;") )   
    from_pages_e.append(explores[e]['x_from_page'])
    

    

In [201]:


get_group(71)

'71-77'

In [202]:

with open("explore_lookup.php", 'w') as f:
    f.write("<?php ")
    t = '$titles_e=["'+'","'.join(titles_e)+'"' + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))
    t = "$explore_files=['"+"','".join(explore_files)+"'" + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))    
    t = "$chapters_e=['"+"','".join(chapters_e)+"'" + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))   
    t = '$subchapters_e=["'+'","'.join(subchapters_e)+'"' + "];\n"
#     print(t)
    f.write(t.encode("utf-8"))   
    t = "$from_pages_e=['"+"','".join(map(str,from_pages_e))+"'" + "];\n"
    f.write(t.encode("utf-8"))   
    f.write(" ?>")

In [203]:
explores[1]

{'a_title': '1.  Chick-a-dee calls of North American chickadees.',
 'chapter': '2. Birds and Their Sounds',
 'subchapter': 'Birds sing and call',
 'x_from_page': 8}

In [204]:

text = '''					<li><a href="#"><i class="fas fa-plus-circle"></i> Explore 1-10</a>
						<ol class="show-hide">
							<li><a href="explore.php">Chick-a-dee calls of North American chickadees.</a></li>
							<li><a href="explore.php">White-breasted nuthatch songs.</a></li>
							<li><a href="explore.php">American robin songs.</a></li>
							<li></li>
							<li></li>
							<li></li>
							<li></li>
							<li></li>
							<li><a href="explore.php">Singing from the nest by cardinals and grosbeaks.</a></li>
							<li><a href="explore.php">Whistling wings of mourning doves.</a></li>
						</ol>
					</li>
'''

explore_text_heading = u'''<li><a href="#" id="{explore_link}"><i class="fas fa-plus-circle"></i> {explore_range}</a>
						<ul class="show-hide">'''

chapter_li = u'''							<li><a href="explore.php?page={page}">{explore_title}</a></li>
'''

explore_heading_end = u'''						</ul>
					</li>
'''

explore_text = ''


def get_group(e):
    start = math.floor((e-1)/10)
    end = int((start+1)*10)

    start = int(start*10+1)
    if end > 70:
        end = 77
    return "{}-{}".format(start,end )

groups = []

for e in explores:
    group = get_group(e)
    if group not in groups:
        groups.append(group)

for group in groups:
    explore_text += explore_text_heading.format(explore_range = group,explore_link=webify_title(group))
    for e in explores:
        if get_group(e)==group:
            explore_text += chapter_li.format(page = e, explore_title = explores[e]['a_title'])
    explore_text += explore_heading_end



            

with open("explore_index.php", 'w') as f:
    f.write(explore_text.encode("utf-8"))
            
# print(explore_text)